# Findings

Plans:
1. Test for amount of journeys in cph
2. Test for avg of zone-travesal 
3. Test for station/stop representations
4. Test for time-specific travels
5. Test for avg travel time

---

1. $20\%$ of journeys in SJWaypoints are within cph. Likewise ~$25\%$ journeys contain a leg of the journey with a REG (regional-tog) train.
2. The average amount of validInternalZones is $13.21$. This is what we expected due to the ticketing system requirement of journeys traversing more than 8 zones must buy a (non-zone) ticket
3. There are $6312$ distinct stop-names in the data. Note: Stations such as Nørrebro will likely occur twice due to the station having both a metro and a s-train. Thus this number is not neccesarily entirely correct. Of these 6312 stops, these are the top 5 names occuring in the table:

|Name | Count|
|----|-----|
|til fods       |20680  |
|København H    |12808  |
|Nørreport St.  |11198  |
|Østerport St.  |8342   |
|Hellerup St.   |8227   |

Top 4 occurances of startStop:

|Id | name| Occurance | 
|---| ---| --- | 
|8600626 | København H.         |1370|
|8600646 | Nørreport St.        |845 |
|8600683 | Hillerød St.         |627 |
|8600714 | Frederikssund St.    |490 |

For top 5 occurances of EndStop, the table is similar to the other table just with different amount of Occurances: 

1. (8600626.0, Kbh H., 1619), 
2. (8600646.0, Nørreport, 791), 
3. (8600683.0, Hillerød, 700), 
4. (8600666.0, Humlebæk , 542),
5. (8600669.0, Helsingør, 517)

Again, remember that København H. is a different station to København H. (Metro) and therefore indicate travels with either regional trains or s-trains. 

By taking a deep dive into which type of waypoint each of the stations use (Looking at Købehavn and Nørreport) we can see that each station mainly occur as a 'waypoint' type indicating that the station is used *as part of the journey* and not as an *active* stop. 


|Name | Type | Count |
| --- | ---  | ----  |
|København H        |IC          |362   |
|    -              |LYN         |29    |
|    -              |REG         |1884  |
|    -              |S           |1307  |
|    -              |TOG         |3     |
|    -              |WALK        |807   |
|    -              |**waypoint**    |**8416**  |
|København H (Metro)|M           |942   |
|    -              |WALK        |431   |
|    -              |waypoint    |1080  |


Name                               Type    
Nørreport St.                      IC            67
                                   LYN            5
                                   REG          905
                                   S           1380
                                   WALK        1290
                                   waypoint    7551
Nørreport St. (Frederiksborggade)  BUS           16
                                   WALK         221
                                   waypoint     302
Nørreport St. (Metro)              M           1771
                                   WALK        1383
                                   waypoint    4673
Nørreport St. (Nørre Voldgade)     BUS          339
                                   EXB          105
                                   NB             9
                                   WALK         520
                                   waypoint    1055




In [15]:
import pandas as pd
import numpy as np
data = pd.read_csv('../Data/JTOSJW_dec_jan_2022.csv')
data.groupby('JId').agg(list).count()

CreatedOn             35948
SearchStart           35948
SearchEnd             35948
StartStop             35948
EndStop               35948
StartZone             35948
Endzone               35948
internalStartZones    35948
internalValidZones    35948
SJId                  35948
Id                    35948
Name                  35948
Latitude              35948
Longitude             35948
Type                  35948
SJSearchJourney_Id    35948
dtype: int64

In [6]:
#Copenhagen filtering
condition_1_cph = (
    (data['internalValidZones'].str.match(r'^(1001|1002|1003|1004)(,(1001|1002|1003|1004))*$')
    | # or
    pd.isna(data['internalValidZones']))
    )

condition_2_cph = (
    (data['internalStartZones'].str.match(r'^(1001|1002|1003|1004)$'))
    | # or
    pd.isna(data['internalStartZones'])
    )

In [7]:
data = data[(condition_1_cph)]
data = data[(condition_2_cph)]
data.groupby('JId').agg(list).count()

/var/folders/6c/gl4j6k8x3tn79jcjxtk_ly1h0000gn/T/ipykernel_45468/4143612775.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data = data[(condition_2_cph)]


CreatedOn             7790
SearchStart           7790
SearchEnd             7790
StartStop             7790
EndStop               7790
StartZone             7790
Endzone               7790
internalStartZones    7790
internalValidZones    7790
SJId                  7790
Id                    7790
Name                  7790
Latitude              7790
Longitude             7790
Type                  7790
SJSearchJourney_Id    7790
dtype: int64

In [172]:
# Step 1: Group by 'JId' and aggregate 'internalValidZones' as a list
grouped_data = data.groupby('JId')['internalValidZones'].agg(list)
avg = 0

count = 0
for SJ_zone in grouped_data:
    avg += len(SJ_zone[0].split(','))
    if len(SJ_zone[0].split(',')) > 13:
        print(SJ_zone[0])
        count = count + 1
print(count)
avg / grouped_data.count()


1044,1055,1054,1043,1032,1033,1067,1066,1053,1042,1065,1064,1031,1002,1077,1076,1075,1052,1041,1063,1062,1074,1030,1001,1003,1089,1088,1087,1008,1086,1085
1043,1054,1042,1031,1032,1033,1044,1055,1053,1066,1065,1064,1052,1041,1030,1002,1067,1063,1062,1077,1076,1075,1074,1051,1061,1050,1040,1001,1003,1073,1072,1071,1089,1088,1087,1008,1086,1085,1084
1004,1003,1001,1002,1032,1043,1054,1066,1076,1008,1097,1028,1027,1141,1140,1269,1262,1260
1041,1051,1050,1040,1030,1031,1042,1053,1052,1061,1060,1062,1002,1032,1043,1054,1065,1064,1063,1071,1070,1072,1073
1018,1016,1017,1019,1092,1038,1039,1036,1011,1012,1037,1010,1009,1091,1082,1093,1034,1069,1049,1014,1013,1090,1081,1080,1071,1072,1083,1084,1094,1045,1035,1068,1006
1170,1177,1114,1110,1111,1115,1029,1096,1008,1076,1066,1054,1043,1032,1002,1001
1180,1145,1140,1141,1027,1021,1020,1099,1089,1077,1067,1055,1044,1033,1002,1001
1005,1015,1013,1080,1070,1060,1050,1040,1030,1002,1001,1032,1043,1054,1066,1076,1008
1004,1003,1001,1002,1030,1031,1032,

13.21044286191165

In [177]:
data[(data['internalValidZones'] == '1008,1086,1075,1076,1087,1097,1096,1095,1085,1074,1064,1065,1066,1067,1077,1088,1098,1028,1029,1048,1047,1094,1084,1073,1063,1053,1054,1055,1089,1099,1020,1021,1027,1142,1141,1115,1111,1046,1034,1093,1083,1007,1045,1072,1062,1052,1041,1042,1043,1044,1023,1022,1026,1146,1277,1143,1140,1114,1116,1117,1110,1079,1092,1082,1035,1071,1078,1061,1051,1050,1040,1030,1031,1032,1033,1024,1025,1273,1275,1262,1269,1274,1144,1145,1176,1177,1113,1184,1185,1181,1112,1038,1018,1019,1009,1081,1006,1080,1070,1060,1002,1272,1056,1270,1263,1264,1260,1268,1267,1261,1276,1180,1137,1138,1170,1171,1165,1131,1132')].groupby('JId').count()

data[(data['JId'] == '08376fbb-e782-4d31-bdbb-1b8d9caa46b3')]

,JId,CreatedOn,SearchStart,SearchEnd,StartStop,EndStop,StartZone,Endzone,internalStartZones,internalValidZones,SJId,Id,Name,Latitude,Longitude,Type,SJSearchJourney_Id
103663,08376fbb-e782-4d31-bdbb-1b8d9caa46b3,2022-12-31 11.57.48.1596868,Roskilde St. (08),Roskilde St. (08),NaN,NaN,0,0,1008,"1008,1086,1075,1076,1087,1097,1096,1095,1085,1...",b5dc0fd4-74d1-488b-902b-fc27ed6ab399,8600624.0,Valby St.,55663993,12514450,waypoint,b4c78b5b-324c-4b5c-a5bf-5325b284c07c
103664,08376fbb-e782-4d31-bdbb-1b8d9caa46b3,2022-12-31 11.57.48.1596868,Roskilde St. (08),Roskilde St. (08),NaN,NaN,0,0,1008,"1008,1086,1075,1076,1087,1097,1096,1095,1085,1...",044f5193-ecfd-4fc2-8aa4-9a97c6b32f0c,8600617.0,Roskilde St.,55639093,12088550,REG,b4c78b5b-324c-4b5c-a5bf-5325b284c07c
103665,08376fbb-e782-4d31-bdbb-1b8d9caa46b3,2022-12-31 11.57.48.1596868,Roskilde St. (08),Roskilde St. (08),NaN,NaN,0,0,1008,"1008,1086,1075,1076,1087,1097,1096,1095,1085,1...",3c8ce979-c1a6-4aa2-bb12-b981e9852faa,8600622.0,Glostrup St.,55663148,12397824,waypoint,b4c78b5b-324c-4b5c-a5bf-5325b284c07c
103666,08376fbb-e782-4d31-bdbb-1b8d9caa46b3,2022-12-31 11.57.48.1596868,Roskilde St. (08),Roskilde St. (08),NaN,NaN,0,0,1008,"1008,1086,1075,1076,1087,1097,1096,1095,1085,1...",6181a155-1980-48c0-b9d0-b4960fb297a2,8600755.0,Trekroner St.,55647929,12132282,waypoint,b4c78b5b-324c-4b5c-a5bf-5325b284c07c
103667,08376fbb-e782-4d31-bdbb-1b8d9caa46b3,2022-12-31 11.57.48.1596868,Roskilde St. (08),Roskilde St. (08),NaN,NaN,0,0,1008,"1008,1086,1075,1076,1087,1097,1096,1095,1085,1...",c5ca9a66-2500-4793-8556-1de01aaa66e2,8600798.0,Høje Taastrup St.,55648873,12269583,waypoint,b4c78b5b-324c-4b5c-a5bf-5325b284c07c
103668,08376fbb-e782-4d31-bdbb-1b8d9caa46b3,2022-12-31 11.57.48.1596868,Roskilde St. (08),Roskilde St. (08),NaN,NaN,0,0,1008,"1008,1086,1075,1076,1087,1097,1096,1095,1085,1...",ead3e6fa-2949-4f81-b3c0-25cfaec0faa6,8600619.0,Hedehusene St.,55648990,12197310,waypoint,b4c78b5b-324c-4b5c-a5bf-5325b284c07c


In [38]:
station_counts = data['Name'].value_counts()
print(station_counts)


Name
Lyngbakken (Frederiksværkvej)         1
Guldnældevænget (Bangsebrovej)        1
Hårlevmark                            1
Hastrupvej (Roskildevej)              1
Fællesvej (Byvej)                     1
                                  ...  
Hellerup St.                       8227
Østerport St.                      8342
Nørreport St.                     11198
København H                       12808
til fods                          20680
Name: count, Length: 6312, dtype: int64


In [126]:
# station_counts = data[['JId', 'StartStop']].groupby('JId').value_counts()
station_counts = data.groupby('JId')['StartStop'].agg(list)
st_count = {}
for st in station_counts:
    if str(st[0]) != "nan": 
        station = st[0]
        if station in st_count:
            st_count.update({station : st_count[station]+1})
        else:
            st_count[station] = 1


top_4_pairs = sorted(st_count.items(), key=lambda item: item[1], reverse=True)[:5]

print(top_4_pairs)
print('----')
print(data[data['Id'] == 8600669.0].Name)



[(8600626.0, 1370), (8600646.0, 845), (8600683.0, 627), (8600666.0, 490), (8600669.0, 416)]
----
1355      Helsingør St.
1448      Helsingør St.
1708      Helsingør St.
2356      Helsingør St.
2493      Helsingør St.
              ...      
569656    Helsingør St.
570659    Helsingør St.
570766    Helsingør St.
570767    Helsingør St.
570826    Helsingør St.
Name: Name, Length: 1344, dtype: object


In [132]:
# station_counts = data[['JId', 'EndStop']].groupby('JId').value_counts()
station_counts = data.groupby('JId')['EndStop'].agg(list)
st_count = {}
for st in station_counts:
    if str(st[0]) != "nan": 
        station = st[0]
        if station in st_count:
            st_count.update({station : st_count[station]+1})
        else:
            st_count[station] = 1


top_4_pairs = sorted(st_count.items(), key=lambda item: item[1], reverse=True)[:5]

print(top_4_pairs)


[(8600626.0, 1619), (8600646.0, 791), (8600683.0, 700), (8600666.0, 542), (8600669.0, 517)]


In [140]:
print('----')
print(data[data['Id'] == 8600669.0].Name)

----
1355      Helsingør St.
1448      Helsingør St.
1708      Helsingør St.
2356      Helsingør St.
2493      Helsingør St.
              ...      
569656    Helsingør St.
570659    Helsingør St.
570766    Helsingør St.
570767    Helsingør St.
570826    Helsingør St.
Name: Name, Length: 1344, dtype: object


In [161]:


filtered_data = data[data['Name'].str.contains("København H")][['Name', 'Type']].groupby(['Name', 'Type'])
print(filtered_data.value_counts())

Name                 Type    
København H          IC           362
                     LYN           29
                     REG         1884
                     S           1307
                     TOG            3
                     WALK         807
                     waypoint    8416
København H (Metro)  M            942
                     WALK         431
                     waypoint    1080
Name: count, dtype: int64


In [162]:
filtered_data = data[data['Name'].str.contains("Nørreport")][['Name', 'Type']].groupby(['Name', 'Type'])
print(filtered_data.value_counts())

Name                               Type    
Nørreport St.                      IC            67
                                   LYN            5
                                   REG          905
                                   S           1380
                                   WALK        1290
                                   waypoint    7551
Nørreport St. (Frederiksborggade)  BUS           16
                                   WALK         221
                                   waypoint     302
Nørreport St. (Metro)              M           1771
                                   WALK        1383
                                   waypoint    4673
Nørreport St. (Nørre Voldgade)     BUS          339
                                   EXB          105
                                   NB             9
                                   WALK         520
                                   waypoint    1055
Name: count, dtype: int64


In [178]:
filtered_data = data[data['Type'].str.contains("REG")][['JId', 'Type']].groupby('JId')
print(filtered_data.value_counts())

JId                                   Type
000503aa-0ba1-4a7a-9be2-09547341f26f  REG     1
0005c862-2721-4cb6-8b95-6c32b17b71c4  REG     1
000c8dbe-0c3b-4e17-ae58-5bf525112ac6  REG     1
000e8a69-1455-4845-bdd7-771b132aad44  REG     1
0016a301-6de3-416e-94fa-42fa79c5fdf3  REG     1
                                             ..
ffbad515-100a-415b-9464-b847db878d5e  REG     1
ffc74bc5-6f60-4c7b-98b3-959c9c191c5d  REG     1
ffdc8d3b-dc4d-4dc7-9b45-14000680491a  REG     1
ffed4bfa-8c95-446f-851b-ae79132904a9  REG     1
fff82616-0809-454f-aee7-0d3b3f5f1472  REG     1
Name: count, Length: 8769, dtype: int64
